In [1]:
import os
import pandas as pd
import datetime as dt
import pytz
import time
import numpy as np
from statistics import mean

from uce_resources import get_applied_forecast

In [2]:

sites_list = ['Pohrebyshche', 'Bilashky', 
                'Bar', 'Balky', 'Verkhivka', 'Kopaihorod', 'Stanislavchyk', 'Sharhorod_1', 'Hnatkiv', 'Porohy',
                'Hlybochok_1', 'Hlybochok_2.1', 'Hlybochok_2.2', 'Cherniatka','Chechelnyk_1', 'Chechelnyk_2',
                'Komyshany_1', 'Komyshany_2', 'Bilozerka', 'Myroliubivka', 'Kyselivka',
                'Oleshky_1', 'Oleshky_2', 'Poniativka', 'Kostohryzove', 
                'Veliton', 'Velihen', 
                'Mykolaivka', 'Rubanivka', 'Mala_Lepetykha', 'Kachkarivka', 
                'Liubymivka', 'Vasylivka',
                'Dibrovka', 'Yelanets_1', 'Yelanets_2', 
                'Afanasiivka', 'Novokondakove', 'Bazaltova', 'Inhulets_1', 'Inhulets_2', 'Bereznehuvate', 
                'Teplychna', 
                'Balivka', 'Solone', 'Stepnohirsk', 
                'Kulevcha']

today = dt.datetime.today()#-dt.timedelta(1)

target_year = today.year
target_month = today.month
target_day = today.day
forecasts_types = ['real']

target_folder = 'data/forecasts/factor/'
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

# sites_list = ['Pohrebyshche', 'Kulevcha']


# Data preparation section

In [3]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.pool import NullPool
from sqlalchemy.sql import select, and_
from settings.db import DO_URL

engine_source = create_engine(DO_URL, poolclass=NullPool)
metadata_source = MetaData()
metadata_source.reflect(bind=engine_source)


from settings.db import WAREHOSUE_URL

engine_warehouse = create_engine(WAREHOSUE_URL, poolclass=NullPool)

C:\Users\n.kozelets\AppData\Local\Temp\ipykernel_3152\1008880204.py:8: SAWarning: Did not recognize type 'point' of column 'location'
  metadata_source.reflect(bind=engine_source)


In [4]:
sites_data = list()

with engine_source.connect() as connection:
        
    for site in sites_list:
        start = time.time()
        site_data = dict()

        sites_table = metadata_source.tables['sites']
        list_to_select = [
            sites_table.c.id]
        
        query = select(list_to_select).where(sites_table.c.displayable_name == site)
        site_id_response = connection.execute(query).fetchall()[0]

        site_id = site_id_response[0]
        
        with engine_warehouse.connect() as connection_warehouse:
            query = f"""
            SELECT grid_capacity from dim_site
            WHERE site_name = '{site}';
            """
            response = connection_warehouse.execute(query).fetchall()[0]
            

        first_date=today-dt.timedelta(1)
        last_date=today

        applied_forecast = get_applied_forecast(
            site_id, 
            first_date,
            last_date,
            connection=connection, 
            db_table=metadata_source.tables['forecasting_data']
        )
        applied_forecast.columns = ['applied_forecast']

        raw_forecast = get_applied_forecast(
            site_id, 
            first_date,
            last_date,
            connection=connection, 
            db_table=metadata_source.tables['forecasting_data'],
            forecast_type='forecast_applied_raw'
        )
        raw_forecast.columns = ['raw_forecast']

        forecast_data = pd.concat([applied_forecast, raw_forecast,], axis=1, join='outer').reindex(columns=['raw_forecast', 'applied_forecast'])
        forecast_data[f"{site}"] = round(forecast_data['applied_forecast'] / forecast_data['raw_forecast'], 2)
        forecast_data = forecast_data.drop(columns=["raw_forecast", "applied_forecast"])
        # print(forecast_data)

        sites_data.append(forecast_data)
        merged_data = pd.concat(sites_data, axis=1)

        end = time.time()

        print('{}: ок! Processing took {} seconds'.format(site, round(end - start, 2)))

    merged_data.index=merged_data.index.tz_localize(pytz.utc).tz_convert(pytz.timezone('europe/kiev')).strftime('%Y-%m-%d %H:%M')
    # print(merged_data)  
    merged_data.to_excel(target_folder + f'forecasting_koef_{target_year}_{target_month}_{target_day}.xlsx')
        


Pohrebyshche: ок! Processing took 1.13 seconds
Bilashky: ок! Processing took 0.51 seconds
Bar: ок! Processing took 0.46 seconds
Balky: ок! Processing took 0.46 seconds
Verkhivka: ок! Processing took 0.45 seconds
Kopaihorod: ок! Processing took 0.57 seconds
Stanislavchyk: ок! Processing took 0.47 seconds
Sharhorod_1: ок! Processing took 0.47 seconds
Hnatkiv: ок! Processing took 0.46 seconds
Porohy: ок! Processing took 0.49 seconds
Hlybochok_1: ок! Processing took 0.47 seconds
Hlybochok_2.1: ок! Processing took 0.45 seconds
Hlybochok_2.2: ок! Processing took 0.44 seconds
Cherniatka: ок! Processing took 0.45 seconds
Chechelnyk_1: ок! Processing took 0.45 seconds
Chechelnyk_2: ок! Processing took 0.45 seconds
Komyshany_1: ок! Processing took 0.45 seconds
Komyshany_2: ок! Processing took 0.47 seconds
Bilozerka: ок! Processing took 0.49 seconds
Myroliubivka: ок! Processing took 0.45 seconds
Kyselivka: ок! Processing took 0.44 seconds
Oleshky_1: ок! Processing took 0.44 seconds
Oleshky_2: ок!